# Ollama template chatbot

## Model

- Phi3-128k-medium

## Imports

In [ ]:
#r "nuget:Scriban"

using System.Net.Http;
using System.Text.Json;
using Scriban;
using Microsoft.DotNet.Interactive;

var client = new HttpClient();


## Supporting classes and records

In [ ]:
public record Message(string role, string content);
public record CompletionResponse(Message message);
public class Chat
{
    public List<Message> History { get; set; }

    public Chat(List<Message> history)
    {
        History = history;
    }    
}

## Supporting methods

In [ ]:
async Task<string> Completion(string prompt, string model = "phi3:medium-128k", string endpoint="http://localhost:11434/api/chat")
{   
    var messages = new Message[] { new Message("user", prompt) };    
    var jsonStr = JsonSerializer.Serialize(new { model,messages,stream=false });
    var response = await client.PostAsync(endpoint, new StringContent(jsonStr, System.Text.Encoding.UTF8, "application/json"));
    if (response.IsSuccessStatusCode)
    {
        var responseStr = await response.Content.ReadAsStringAsync();
        var completion = JsonSerializer.Deserialize<CompletionResponse>(responseStr);
        return completion.message.content;
    }
    return string.Empty;
}

string RenderTemplate(string template, object model)
{
    var templateObject = Template.Parse(template);
    var result = templateObject.Render(model);
    return result;
}

## Prepare the chat history and template

In [ ]:
var chat = new Chat([new Message("system", "You are a grumpy but funny assistant.")]);
var template = "{{ for message in history }}{{ message.role }}:\n{{ message.content }}\n{{ end }}\nuser:\n{{prompt}}";

## Process the user's input while keeping the history

In [ ]:
while(true)
{
    var input = await Kernel.GetInputAsync("Enter a your prompt: ");
    if (input == "exit")
    {
        break;
    }
    var finalPrompt = RenderTemplate(template, new { history = chat.History, prompt = input });
    Console.WriteLine($"Q: {input}");
    chat.History.Add(new Message("user", input));
    var resp = await Completion(finalPrompt);
    Console.WriteLine($"A: {resp}");
    chat.History.Add(new Message("assistant", resp));
    // As the history grows, we should limit the number of messages to keep within the model's context window
    // Refer to this blog: https://blog.pamelafox.org/2024/06/truncating-conversation-history-for.html
}